# Serverless Task Migration
This app is designed to help identify tasks running in your snowflake that are underutilizing a warehouse so that they can be changed to a serverless task. This is designed to migrate these tasks to improve the proficiency of these services as well as save on the cost of execution, as an under-utilized warehouse often charges for downtime when spun up for short tasks

This Notedbook will drop a streamlit file at SIT_SOLUTIONS.STM.CODE_STAGE and install the app SIT_SOLUTIONS.STM.STM_STREAMLIT.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql(f"""ALTER SESSION SET QUERY_TAG = '{{"origin":"sf_sit","name":"sit_serveless_task_migration","version":{{"major":1, "minor":0}} }}'""").collect()

current_database = session.sql("SELECT CURRENT_DATABASE()").collect()[0][0]
current_schema = session.sql("SELECT CURRENT_schema()").collect()[0][0]
try:
    # session.sql("SELECT * from INSTALL_TEST.TEST.TEST_OPERATIONS").collect()
    st.session_state.opps_test_info = session.sql(f"SELECT * from {current_database}.{current_schema}.TEST_OPERATIONS").to_pandas()
    run_mode = "test"
except:
    st.success("Running Customer Install. Please follow the steps!")
    run_mode = "customer"

Please choose the location you'd like to install to

In [ ]:
if run_mode == 'test':
    opps_info = st.session_state.opps_test_info
    install_db = opps_info[opps_info["KEY"] == "INSTALL_DB"]["VALUE"].values[0]
    admin_schema = opps_info[opps_info["KEY"] == "ADMIN_SCHEMA"]["VALUE"].values[0]
    results_schema = opps_info[opps_info["KEY"] == "RESULT_SCHEMA"]["VALUE"].values[0]
    ops_schema = opps_info[opps_info["KEY"] == "OPS_SCHEMA"]["VALUE"].values[0]
    install_stage = opps_info[opps_info["KEY"] == "INSTALL_STAGE"]["VALUE"].values[0]
    ops_wh = opps_info[opps_info["KEY"] == "OPS_WH"]["VALUE"].values[0]
elif run_mode =="customer":
    install_db = st.text_input("Database")
    admin_schema = st.text_input("Admin Schema")
    results_schema = st.text_input("Results Schema")
    ops_schema = st.text_input("Operations Schema")
    install_stage = st.text_input("Install Stage (for hosting code files)")
    ops_wh = st.text_input("User Warehouse")

In [ ]:
CREATE OR REPLACE PROCEDURE {{install_db}}.{{admin_schema}}.PUT_TO_STAGE(STAGE VARCHAR, FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.8
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;


In [ ]:
"""gASVcRoAAAAAAAB9lIwFZmlsZXOUfZQojAguZ2l0a2VlcJRDAJSMD2Vudmlyb25tZW50LnltbJRD
Q25hbWU6IHNmX2VudgpjaGFubmVsczoKLSBzbm93Zmxha2UKZGVwZW5kZW5jaWVzOgotIHN0cmVh
bWxpdD0xLjM5LjCUjBRpbnN0YWxsX3NvbHV0aW9uLnNxbJRC0gIAAC0tIWppbmphMgoKQ1JFQVRF
IE9SIFJFUExBQ0UgVEFCTEUge3sgaW5zdGFsbF9kYiB9fS57eyBvcHNfc2NoZW1hIH19LlRFU1Rf
T1BFUkFUSU9OUyhLRVkgU1RSSU5HLCBWQUxVRSBTVFJJTkcpOwoKSU5TRVJUIElOVE8ge3sgaW5z
dGFsbF9kYiB9fS57eyBvcHNfc2NoZW1hIH19LlRFU1RfT1BFUkFUSU9OUwpWQUxVRVMKKCdJTlNU
QUxMX0RCJywne3sgaW5zdGFsbF9kYiB9fScpLAooJ0FETUlOX1NDSEVNQScsJ3t7IGFkbWluX3Nj
aGVtYSB9fScpLAooJ09QU19TQ0hFTUEnLCd7eyBvcHNfc2NoZW1hIH19JyksCignUkVTVUxUX1ND
SEVNQScsJ3t7IHJlc3VsdHNfc2NoZW1hIH19JyksCignSU5TVEFMTF9TVEFHRScsJ3t7IGluc3Rh
bGxfc3RhZ2UgfX0nKSwKKCdPUFNfV0gnLCd7eyBvcHNfd2ggfX0nKTsKCkNSRUFURSBPUiBSRVBM
QUNFIFNUUkVBTUxJVCB7eyBpbnN0YWxsX2RiIH19Lnt7IG9wc19zY2hlbWEgfX0uU1RNX1NUUkVB
TUxJVApST09UX0xPQ0FUSU9OID0gJ0B7e2luc3RhbGxfZGJ9fS57e29wc19zY2hlbWF9fS57e2lu
c3RhbGxfc3RhZ2V9fScKTUFJTl9GSUxFID0gJ3NlcnZlcmxlc3NfdGFza19taWdyYXRpb24ucHkn
ClFVRVJZX1dBUkVIT1VTRSA9IHt7b3BzX3dofX0KQ09NTUVOVD0neyJvcmlnaW4iOiAic2Zfc2l0
IiwibmFtZSI6ICJzaXRfc2VydmVsZXNzX3Rhc2tfbWlncmF0aW9uIiwidmVyc2lvbiI6ICJ7bWFq
b3I6IDEsIG1pbm9yOiAwfSJ9JzsKlIwcc2VydmVybGVzc190YXNrX21pZ3JhdGlvbi5weZRC5xYA
ACMgSW1wb3J0IHB5dGhvbiBwYWNrYWdlcwppbXBvcnQgc3RyZWFtbGl0IGFzIHN0CmltcG9ydCBw
YW5kYXMgYXMgcGQKaW1wb3J0IHRpbWUKZnJvbSBzbm93Zmxha2Uuc25vd3BhcmsuY29udGV4dCBp
bXBvcnQgZ2V0X2FjdGl2ZV9zZXNzaW9uCgpzdC5zZXRfcGFnZV9jb25maWcobGF5b3V0PSJ3aWRl
IikKIyBXcml0ZSBkaXJlY3RseSB0byB0aGUgYXBwCnN0LmltYWdlKCdodHRwczovL3Jhdy5naXRo
dWJ1c2VyY29udGVudC5jb20vU25vd2ZsYWtlLUxhYnMvZW1lcmdpbmctc29sdXRpb25zLXRvb2xi
b3gvbWFpbi9iYW5uZXJfZW1lcmdpbmdfc29sdXRpb25zX3Rvb2xib3gucG5nJykKCnN0LnRpdGxl
KCJTZXJ2ZXJsZXNzIFRhc2tzIE1pZ3JhdGlvbiIpCiMgR2V0IHRoZSBjdXJyZW50IGNyZWRlbnRp
YWxzCnNlc3Npb24gPSBnZXRfYWN0aXZlX3Nlc3Npb24oKQoKY3NzID0gIiIiCjxzdHlsZT4KICAg
IC8qVVNFUyBUSEUgSEFTIFNFTEVDVE9SIFRPIEZJTFRFUiBCWSBUSEUgTkVTVEVEIENPTlRBSU5F
UiovCgpkaXZbcm9sZT0iZGlhbG9nIl06aGFzKC5zdC1rZXktbWlncmF0ZV9jb250YWluZXIpewog
ICAgd2lkdGg6ODUlOwp9Cjwvc3R5bGU+IiIiCnN0Lmh0bWwoY3NzKQoKCkBzdC5kaWFsb2coIk1p
Z3JhdGUgVGFzayIpCmRlZiBtaWdyYXRlX3Rhc2sodGFza19uYW1lLCB3aF9uYW1lLCB3aF9zaXpl
KToKICAgIHdpdGggc3QuY29udGFpbmVyKGtleT0ibWlncmF0ZV9jb250YWluZXIiKToKICAgICAg
ICBzdC53cml0ZShmIlRoaXMgTWlncmF0aW9uIHdpbGwgcmVtb3ZlIHRhc2sge3Rhc2tfbmFtZX0i
KQogICAgICAgIHN0LndyaXRlKGYiZnJvbSB3YXJlaG91c2Uge3doX25hbWV9LCBhbmQgbWFrZSBp
dCBzZXJ2ZXJsZXNzIHdpdGggdGhlIGJlbG93IHNldHRpbmciKQogICAgICAgIHdoX3NpemVfbGlz
dCA9IFsiWFNNQUxMIiwiU01BTEwiLCJNRURJVU0iLCJMQVJHRSIsIlhMQVJHRSIsIlhYTEFSR0Ui
LCJYWFhMQVJHRSJdCiAgICAgICAgd2hfc2l6ZSA9IHN0LnNlbGVjdGJveCgiSW5pdGlhbCBXYXJl
aG91c2UgU2l6ZSIsd2hfc2l6ZV9saXN0LGluZGV4PXdoX3NpemVfbGlzdC5pbmRleCh3aF9zaXpl
LnJlcGxhY2UoJy0nLCcnKS51cHBlcigpKSkKICAgICAgICBfXyxjb2wyID0gc3QuY29sdW1ucygo
MTAsMikpCiAgICAgICAgaWYgY29sMi5idXR0b24oIk1pZ3JhdGUiLCB0eXBlPSJwcmltYXJ5Iik6
CiAgICAgICAgICAgIHNlc3Npb24uc3FsKGYiQUxURVIgVEFTSyB7dGFza19uYW1lfSBTVVNQRU5E
IikuY29sbGVjdCgpCiAgICAgICAgICAgIHNlc3Npb24uc3FsKGYiQUxURVIgVEFTSyB7dGFza19u
YW1lfSBVTlNFVCBXQVJFSE9VU0UiKS5jb2xsZWN0KCkKICAgICAgICAgICAgc2Vzc2lvbi5zcWwo
ZiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFNFVCBVU0VSX1RBU0tfTUFOQUdFRF9JTklUSUFMX1dB
UkVIT1VTRV9TSVpFID0gJ1hTTUFMTCc7IikuY29sbGVjdCgpCiAgICAgICAgICAgIHNlc3Npb24u
c3FsKGYiIiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFNFVCBDT01NRU5UID0gJ3t7Im9yaWdpbiI6
ICJzZl9zaXQiLCJuYW1lIjogInNpdF9zZXJ2ZWxlc3NfdGFza19taWdyYXRpb24iLCJ2ZXJzaW9u
IjogInt7bWFqb3I6IDEsIG1pbm9yOiAwfX0ifX0nIiIiKS5jb2xsZWN0KCkKICAgICAgICAgICAg
c2Vzc2lvbi5zcWwoZiJBTFRFUiBUQVNLIHt0YXNrX25hbWV9IFJFU1VNRSIpLmNvbGxlY3QoKQog
ICAgICAgICAgICBzdC5zdWNjZXNzKGYie3Rhc2tfbmFtZX0gTWlncmF0ZWQhIikKICAgICAgICAg
ICAgdGltZS5zbGVlcCg1KQogICAgICAgICAgICBzdC5yZXJ1bigpCiAgICAgICAgICAgIAoKQHN0
LmNhY2hlX2RhdGEKZGVmIGdldF93YXJlaG91c2VzKHdoX25hbWVzKToKICAgIHJldHVybiBzZXNz
aW9uLnNxbChmIiIiCiAgICBTRUxFQ1QgRElTVElOQ1QgV0FSRUhPVVNFX05BTUUsIENPTkNBVCh0
LkRBVEFCQVNFX05BTUUsJy4nLHQuU0NIRU1BX05BTUUsJy4nLCB0Lk5BTUUpIGFzIEZVTExfTkFN
RSwgdC5uYW1lIEZST00gU05PV0ZMQUtFLkFDQ09VTlRfVVNBR0UuVEFTS19ISVNUT1JZIHQKICBK
T0lOIFNOT1dGTEFLRS5BQ0NPVU5UX1VTQUdFLlFVRVJZX0hJU1RPUlkgcSBPTiB0LlFVRVJZX0lE
ID0gcS5RVUVSWV9JRAogIFJJR0hUIEpPSU4gU05PV0ZMQUtFLkFDQ09VTlRfVVNBR0UuTUVURVJJ
TkdfSElTVE9SWSB3IE9OIHEud2FyZWhvdXNlX25hbWUgPSB3Lm5hbWUKICBXSEVSRSAKICBRVUVS
WV9TVEFSVF9USU1FIEJFVFdFRU4gdy5TVEFSVF9USU1FIEFORCB3LkVORF9USU1FCiAgQU5EIE5P
VCBDT05UQUlOUyhXQVJFSE9VU0VfTkFNRSwnQ09NUFVURV9TRVJWSUNFJykgCiAgQU5EIFNDSEVE
VUxFRF9USU1FIEJFVFdFRU4gZGF0ZWFkZChEQVksIC0gMTQsIENVUlJFTlRfVElNRVNUQU1QKCkp
IEFORCBDVVJSRU5UX1RJTUVTVEFNUCgpCiAgQU5EIHcubmFtZSBpbiAoe3doX25hbWVzfSk7CiAg
ICAiIiIpLnRvX3BhbmRhcygpCgppZiAicXVhbF90YXNrcyIgbm90IGluIHN0LnNlc3Npb25fc3Rh
dGU6CiAgICB3aXRoIHN0LnNwaW5uZXIoIkZldGNoaW5nIFRhc2tzIik6CgogICAgICAgIHN0LnNl
c3Npb25fc3RhdGUucXVhbF90YXNrcyA9IHNlc3Npb24uc3FsKCIiIlNFTEVDVCBESVNUSU5DVCBD
T05DQVQodC5EQVRBQkFTRV9OQU1FLCcuJyx0LlNDSEVNQV9OQU1FLCcuJywgTkFNRSkgYXMgRlVM
TF9OQU1FLE5BTUUsIHQuUVVFUllfVEVYVCwgV0FSRUhPVVNFX05BTUUsIFdBUkVIT1VTRV9TSVpF
LCBpZmYoREFURURJRkYoIk1JTlVURSIsUVVFUllfU1RBUlRfVElNRSwgQ09NUExFVEVEX1RJTUUp
IDwgMSwgJ1RSVUUnLCdGQUxTRScpIGFzIFNIT1JUX1JVTlRJTUUsIGlmZihEQVRFRElGRigiTUlO
VVRFIixRVUVSWV9TVEFSVF9USU1FLCBTQ0hFRFVMRURfVElNRSkgPiAxLCAnVFJVRScsJ0ZBTFNF
JykgYXMgUkFOX09WRVIgRlJPTSBTTk9XRkxBS0UuQUNDT1VOVF9VU0FHRS5UQVNLX0hJU1RPUlkg
dAogICAgICAgICAgSk9JTiBTTk9XRkxBS0UuQUNDT1VOVF9VU0FHRS5RVUVSWV9ISVNUT1JZIHEg
T04gdC5RVUVSWV9JRCA9IHEuUVVFUllfSUQKICAgICAgICAgIFdIRVJFIAogICAgICAgICAgKERB
VEVESUZGKCJNSU5VVEUiLFFVRVJZX1NUQVJUX1RJTUUsIENPTVBMRVRFRF9USU1FKSA8IDEKICAg
ICAgICAgIE9SCiAgICAgICAgICBEQVRFRElGRigiTUlOVVRFIixRVUVSWV9TVEFSVF9USU1FLCBT
Q0hFRFVMRURfVElNRSkgPiAxKQogICAgICAgICAgQU5EIE5PVCBDT05UQUlOUyhXQVJFSE9VU0Vf
TkFNRSwnQ09NUFVURV9TRVJWSUNFJykgCiAgICAgICAgICBBTkQgU0NIRURVTEVEX1RJTUUgQkVU
V0VFTiBkYXRlYWRkKERBWSwgLSAxNCwgQ1VSUkVOVF9USU1FU1RBTVAoKSkgQU5EIENVUlJFTlRf
VElNRVNUQU1QKCk7IiIiKS50b19wYW5kYXMoKQoKcXVhbF90YXNrcyA9IHN0LnNlc3Npb25fc3Rh
dGUucXVhbF90YXNrcwoKd2l0aCBzdC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgc3Qud3Jp
dGUoIiIiVGhlIHRhc2tzIGJlbG93IGFyZSB0aGUgZmlyc3QgZ2xhbmNlIG9mIHRhc2tzIHRoYXQg
bWF5IHF1YWxpZnkgZm9yIG1pZ3JhdGlvbiB0byBzZXJ2ZXJsZXNzIHRhc2tzIFxuCiAgICBUaGV5
IGVpdGhlciBydW4gZm9yIHVuZGVyIGEgbWludXRlLCBvciBydW4gbG9uZ2VyIHRoYW4gdGhlIHRp
bWUgYmV0d2VlbiB0aGVpciBzY2hlZHVsZWQgcnVuIGFuZCB0aGUgbmV4dCBydW4KICAgICIiIikK
ICAgIHN0LmRhdGFmcmFtZShxdWFsX3Rhc2tzLCB1c2VfY29udGFpbmVyX3dpZHRoPVRydWUsIGNv
bHVtbl9jb25maWc9eydGVUxMX05BTUUnOk5vbmV9KQoKd2l0aCBzdC5jb250YWluZXIoYm9yZGVy
PVRydWUpOgogICAgc3Qud3JpdGUoIiIiVGhlIFRhc2tzIHRoYXQgcnVuIGxvbmcgaW4gdGhlIHBy
ZXZpb3VzIGxpc3QgYXJlIGdyZWF0IGNhbmRpZGF0ZXMgZm9yIHNlcnZlcmxlc3Mgd2FyZWhvdXNl
cyBhcyB0aGV5IGNhbiBoYXZlIGEgdGFyZ2V0IGZpbmlzaCBzZXQgYW5kIHRoZSB3YXJlaG91c2Ug
d2lsbCBzY2FsZSBmb3IgdGhlIHNjaGVkdWxlIiIiKQogICAgc3Qud3JpdGUoIiIiRm9yIHNob3J0
IHJ1bm5pbmcgdGFza3MsIHRvIGZ1cnRoZXIgcXVhbGlmeSB0aGVtIHdlJ2xsIGNoZWNrIGhvdyBt
YW55IHRhc2tzIGdlbmVyYWxseSBydW4gb24gdGhlIHNhbWUgd2FyZWhvdXNlIiIiKQoKd2l0aCBz
dC5jb250YWluZXIoYm9yZGVyPVRydWUpOgogICAgc3Qud3JpdGUoIiIiVGhlIGJlbG93IGxpc3Qg
c2hvd3MgdGhlIG51bWJlciBvZiB0YXNrcyB0aGF0IGFsbCBydW4gYXJvdW5kIHRoZSBzYW1lIHRp
bWUgb24gZWFjaCB3YXJlaG91c2UiIiIpCgogICAgd2FyZWhvdXNlcyA9IHNldChxdWFsX3Rhc2tz
WyJXQVJFSE9VU0VfTkFNRSJdLnRvbGlzdCgpKQogICAgd2FyZWhvdXNlc19sID0gIid7fSciLmZv
cm1hdCgiJywgJyIuam9pbih3YXJlaG91c2VzKSkKCiAgICB3YXJlaG91c2VfbGlzdCA9IGdldF93
YXJlaG91c2VzKHdhcmVob3VzZXNfbCkKICAgIHN0LmRhdGFmcmFtZSh3YXJlaG91c2VfbGlzdCx1
c2VfY29udGFpbmVyX3dpZHRoPVRydWUpCgogICAgc3Qud3JpdGUoIkZvciBhbnkgdGFza3MgdGhh
dCBydW4gZm9yIGxlc3MgdGhhbiBhIG1pbnV0ZSBvbiBhIHdhcmVob3VzZSBhbG9uZSBzaG91bGQg
YmUgbWlncmF0ZWQgdG8gYSBzZXJ2ZXJsZXNzIHdhcmVob3VzZSIpCiAgICBzdC53cml0ZSgiQWRk
aXRpb25hbGx5IGFueSB0YXNrcyBzY2hlZHVsZWQgd2l0aCBhIHRvdGFsIHJ1bnRpbWUgb2YgbGVz
cyB0aGFuIGEgZmV3IG1pbnV0ZXMgd291bGQgbGlrZWx5IGVhY2ggYmVuZWZpdCBmcm9tIGJlaW5n
IHNlcnZlcmxlc3MiKQogICAgc3Qud3JpdGUoIlJldmlldyBlYWNoIHdhcmVob3VzZSBiZWxvdyBh
bmQgbWlncmF0ZSBhbnkgdGFza3MgdGhhdCBhcmUgcmVjb21tZW5kZWQgb3IgdGhhdCB5b3Ugd291
bGQgbGlrZSB0byIpCiAgICAKICAgIGZvciB3aF9uYW1lIGluIHdhcmVob3VzZXM6CiAgICAgICAg
d2hfdGFza3MgPSB3YXJlaG91c2VfbGlzdFt3YXJlaG91c2VfbGlzdFsiV0FSRUhPVVNFX05BTUUi
XSA9PSB3aF9uYW1lXVsiTkFNRSJdLnRvbGlzdCgpCiAgICAgICAgd2hfc2l6ZSA9IHF1YWxfdGFz
a3NbcXVhbF90YXNrc1siV0FSRUhPVVNFX05BTUUiXSA9PSB3aF9uYW1lXVsiV0FSRUhPVVNFX1NJ
WkUiXS52YWx1ZXNbMF0KICAgICAgICB3aXRoIHN0LmV4cGFuZGVyKHdoX25hbWUpOgogICAgICAg
ICAgICBzdC53cml0ZSgiIyMjIikKICAgICAgICAgICAgY29sMSxfXyxjb2wyID0gc3QuY29sdW1u
cygzKQogICAgICAgICAgICBjb2wxLnN1YmhlYWRlcigiKlRhc2sgTmFtZSoiKQoKICAgICAgICAg
ICAgZm9yIHdoX3Rhc2sgaW4gd2hfdGFza3M6CiAgICAgICAgICAgICAgICBmdWxsX3Rhc2tfbmFt
ZSA9IHdhcmVob3VzZV9saXN0W3dhcmVob3VzZV9saXN0WyJOQU1FIl0gPT0gd2hfdGFza11bIkZV
TExfTkFNRSJdLnZhbHVlc1swXQogICAgICAgICAgICAgICAgZGVzY3JpYmVfdGFzayA9IHBkLkRh
dGFGcmFtZShzZXNzaW9uLnNxbChmIkRFU0NSSUJFIFRBU0sge2Z1bGxfdGFza19uYW1lfSIpLmNv
bGxlY3QoKSkKICAgICAgICAgICAgICAgIGlmIGRlc2NyaWJlX3Rhc2tbIndhcmVob3VzZSJdLnZh
bHVlc1swXToKICAgICAgICAgICAgICAgICAgICBzdC5kaXZpZGVyKCkKICAgICAgICAgICAgICAg
ICAgICBjb2wxLF9fLGNvbDIgPSBzdC5jb2x1bW5zKDMpCiAgICAgICAgICAgICAgICAgICAgY29s
MS53cml0ZSh3aF90YXNrKQogICAgICAgICAgICAgICAgICAgIGNvbDIuYnV0dG9uKCJNaWdyYXRl
Iixvbl9jbGljaz1taWdyYXRlX3Rhc2ssIGFyZ3M9KGZ1bGxfdGFza19uYW1lLHdoX25hbWUsd2hf
c2l6ZSksIGtleT13aF90YXNrKyJtaWdyYXRlX2J1dHRvbiIpCiAgICAgICAgCiAgICCUdXMu
"""

In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)


# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call {db}.{schema}.put_to_stage('{db}.{schema}.{stage}','{file_name}', $${file_content}$$)""".format(
        file_name=file_name, 
        file_content=file_content,
        db=install_db,
        schema=admin_schema,
        stage=install_stage)
    session.sql(sql).collect()

# Stage each image

In [ ]:
CREATE OR REPLACE STREAMLIT {{install_db}}.{{ops_schema}}.STM_STREAMLIT
ROOT_LOCATION = '@{{install_db}}.{{ops_schema}}.{{install_stage}}'
MAIN_FILE = 'serverless_task_migration.py'
QUERY_WAREHOUSE = {{ops_wh}}
COMMENT='{"origin": "sf_sit","name": "sit_serveless_task_migration","version": "{major: 1, minor: 0}"}';